In [2]:
import numpy as np
import faiss
import pandas as pd
import ast
import time
import optuna

c:\Users\Carlo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embeddings = pd.read_csv(r'C:\Users\Carlo\Desktop\Proyectos\data\bubbo-data\FAISS\all_content_embeddings.csv')

In [5]:
embeddings

,ID,Embedding
0,100,"[-0.08247800171375275, 0.0059804837219417095, ..."
1,10000017,"[-0.02536494843661785, -0.07838041335344315, 0..."
2,10000042,"[0.08953297138214111, 0.08901918679475784, -0...."
3,10000064,"[0.00326346792280674, 0.08406837284564972, -0...."
4,100001,"[-0.03719714283943176, -0.03154193237423897, -..."
...,...,...
141052,tt9795174,"[-0.06088811904191971, -0.043131303042173386, ..."
141053,tt9802804,"[-0.024917157366871834, 0.03381925821304321, -..."
141054,tt9814836,"[-0.03835444897413254, -0.0341656431555748, -0..."
141055,tt9820556,"[-0.07445351779460907, -0.02916163019835949, -..."


In [6]:
embeddings.shape

(141057, 2)

In [7]:
embeddings = np.array([ast.literal_eval(e) for e in embeddings['Embedding']])

In [8]:
embeddings.shape

(141057, 384)

In [10]:
# Dimensiones de los embeddings (ya lo tienes: 384)
d = 384

# Número de embeddings (ya lo tienes: 103000)
n = 102832

# Lista de índices a probar
index_types = [
    "FlatL2",
    "IVFPQ",
    "HNSW"
]

# Parámetros para IVFPQ
nlist = 100
m = 16

# Parámetros para HNSW
M = 32
efConstruction = 200

# Número de consultas de búsqueda
nq = 1000

# Generar consultas aleatorias
query_embeddings = np.random.rand(nq, d).astype('float32')

# Resultados
results = {}

# Iterar sobre los tipos de índices
for index_type in index_types:
    print(f"Probando índice: {index_type}")
    
    # Crear índice
    if index_type == "FlatL2":
        index = faiss.IndexFlatL2(d)
    elif index_type == "IVFPQ":
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
        index.train(embeddings)
    elif index_type == "HNSW":
        index = faiss.IndexHNSWFlat(d, M)
        index.hnsw.efConstruction = efConstruction
    
    # Añadir embeddings al índice
    index.add(embeddings)

    # Medir tiempo de búsqueda
    start_time = time.time()
    D, I = index.search(query_embeddings, k=10)  # Buscar los 10 vecinos más cercanos
    search_time = time.time() - start_time

    # Guardar resultados
    results[index_type] = {
        "search_time": search_time,
        # Aquí puedes añadir métricas de precisión si tienes datos de ground truth
    }

# Imprimir resultados
for index_type, result in results.items():
    print(f"Índice: {index_type}")
    print(f"  Tiempo de búsqueda: {result['search_time']:.4f} segundos")
    # Imprimir métricas de precisión si están disponibles

# Encontrar el mejor índice
best_index = min(results, key=lambda x: results[x]['search_time'])
print(f"\nEl mejor índice es: {best_index}")

Probando índice: FlatL2
Probando índice: IVFPQ
Probando índice: HNSW
Índice: FlatL2
  Tiempo de búsqueda: 0.2581 segundos
Índice: IVFPQ
  Tiempo de búsqueda: 0.0130 segundos
Índice: HNSW
  Tiempo de búsqueda: 0.0310 segundos

El mejor índice es: IVFPQ


In [ ]:
def objective(trial):
    nlist = trial.suggest_int('nlist', 50, 500)  # Rango para nlist
    m_choices =  [8, 12, 16, 24, 32]# Opciones válidas para m (divisores de 384)
    m = trial.suggest_categorical('m', m_choices)

    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
    index.train(embeddings)
    index.add(embeddings)

    start_time = time.time()
    _, _ = index.search(query_embeddings, k=10)
    search_time = time.time() - start_time

    return search_time

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f"Mejores hiperparámetros: {study.best_params}")

[I 2025-02-12 14:54:21,888] A new study created in memory with name: no-name-1fdfd049-d717-42fa-b31e-b83590a3ff40
[I 2025-02-12 14:54:25,559] Trial 0 finished with value: 0.006002664566040039 and parameters: {'nlist': 396, 'm': 8}. Best is trial 0 with value: 0.006002664566040039.
[I 2025-02-12 14:54:28,316] Trial 1 finished with value: 0.005513429641723633 and parameters: {'nlist': 113, 'm': 8}. Best is trial 1 with value: 0.005513429641723633.
[I 2025-02-12 14:54:31,299] Trial 2 finished with value: 0.01000523567199707 and parameters: {'nlist': 180, 'm': 24}. Best is trial 1 with value: 0.005513429641723633.
[I 2025-02-12 14:54:34,374] Trial 3 finished with value: 0.006004810333251953 and parameters: {'nlist': 224, 'm': 8}. Best is trial 1 with value: 0.005513429641723633.
[I 2025-02-12 14:54:37,692] Trial 4 finished with value: 0.00500035285949707 and parameters: {'nlist': 135, 'm': 12}. Best is trial 4 with value: 0.00500035285949707.
[I 2025-02-12 14:54:42,099] Trial 5 finished wi

Mejores hiperparámetros: {'nlist': 375, 'm': 32}


In [8]:
# Dimensiones de los embeddings
d = 384  

# Número de embeddings
n = 102832  

# Generar embeddings aleatorios (simulación)
embeddings = np.random.rand(n, d).astype('float32')

# Lista de índices a probar
index_types = ["FlatL2", "IVFPQ", "HNSW"]

# Parámetros para IVFPQ
nlist = 100
m = 16

# Parámetros para HNSW
M = 32
efConstruction = 200

# Número de consultas de búsqueda
nq = 1000  

# Generar consultas aleatorias
query_embeddings = np.random.rand(nq, d).astype('float32')

# Resultados
results = {}

# 🔹 **Paso 1: Obtener Ground Truth con IndexFlatL2**
index_gt = faiss.IndexFlatL2(d)  
index_gt.add(embeddings)

# Buscar los 10 vecinos más cercanos con el índice exacto
_, I_gt = index_gt.search(query_embeddings, k=10)  

# 🔹 **Paso 2: Evaluar cada índice**
for index_type in index_types:
    print(f"Probando índice: {index_type}")

    # Crear índice
    if index_type == "FlatL2":
        index = faiss.IndexFlatL2(d)
    elif index_type == "IVFPQ":
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
        index.train(embeddings)
    elif index_type == "HNSW":
        index = faiss.IndexHNSWFlat(d, M)
        index.hnsw.efConstruction = efConstruction

    # Añadir embeddings al índice
    index.add(embeddings)

    # Medir tiempo de búsqueda
    start_time = time.time()
    _, I = index.search(query_embeddings, k=10)  
    search_time = time.time() - start_time

    # 🔹 **Paso 3: Calcular Precisión**
    def precision_at_k(I_pred, I_gt, k=10):
        correct = 0
        for i in range(len(I_pred)):
            correct += len(set(I_pred[i]) & set(I_gt[i])) / k
        return correct / len(I_pred)

    precision = precision_at_k(I, I_gt, k=10)

    # Guardar resultados
    results[index_type] = {
        "precision": precision,
        "search_time": search_time
    }

# 🔹 **Paso 4: Imprimir resultados**
for index_type, result in results.items():
    print(f"Índice: {index_type}")
    print(f"  Precisión: {result['precision']:.4f}")
    print(f"  Tiempo de búsqueda: {result['search_time']:.4f} segundos\n")

# 🔹 **Paso 5: Encontrar el índice más preciso**
best_index = max(results, key=lambda x: results[x]['precision'])
print(f"\n🔹 El índice más preciso es: {best_index}")
print(f"   Precisión: {results[best_index]['precision']:.4f}")
print(f"   Tiempo de búsqueda: {results[best_index]['search_time']:.4f} segundos")


Probando índice: FlatL2
Probando índice: IVFPQ
Probando índice: HNSW
Índice: FlatL2
  Precisión: 1.0000
  Tiempo de búsqueda: 0.2229 segundos

Índice: IVFPQ
  Precisión: 0.0107
  Tiempo de búsqueda: 0.0100 segundos

Índice: HNSW
  Precisión: 0.1502
  Tiempo de búsqueda: 0.0442 segundos


🔹 El índice más preciso es: FlatL2
   Precisión: 1.0000
   Tiempo de búsqueda: 0.2229 segundos


In [10]:
import numpy as np
import faiss
import time
import optuna

# Dimensiones de los embeddings
d = 384  # Dimensiones de los embeddings

# Número de embeddings
n = 102832  # Número de embeddings

# Generar embeddings aleatorios (simulación)
embeddings = np.random.rand(n, d).astype('float32')

# Generar consultas aleatorias
nq = 1000
query_embeddings = np.random.rand(nq, d).astype('float32')

# Índices a probar
index_types = ["IVFPQ", "HNSW"]

# 🔹 **Función objetivo para Optuna:**
def objective(trial):
    # **Hiperparámetros para IVFPQ**
    index_type = trial.suggest_categorical("index_type", index_types)
    
    if index_type == "IVFPQ":
        nlist = trial.suggest_int("nlist", 100, 1000)  # Número de listas
        m_choices = [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 384]
        m = trial.suggest_categorical("m", m_choices)  # Número de divisiones
        n_bits = trial.suggest_int("n_bits", 8, 16)  # Bits por cada subvector
        
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFPQ(quantizer, d, nlist, m, n_bits)
        index.train(embeddings)  # Entrenar el índice con los embeddings
    elif index_type == "HNSW":
        M = trial.suggest_int("M", 32, 128)  # Vecinos por nodo
        efConstruction = trial.suggest_int("efConstruction", 100, 400)  # Precisión de construcción
        
        index = faiss.IndexHNSWFlat(d, M)
        index.hnsw.efConstruction = efConstruction
        index.hnsw.efSearch = trial.suggest_int("efSearch", 50, 200)  # Precisión de búsqueda
    else:  # FlatL2 no tiene parámetros para optimizar
        index = faiss.IndexFlatL2(d)
    
    # Añadir embeddings al índice en lotes para evitar problemas de memoria
    batch_size = 10000  # Tamaño de cada lote, puedes ajustarlo
    for i in range(0, n, batch_size):
        batch = embeddings[i:i + batch_size]
        index.add(batch)
    
    # **Obtener Ground Truth con IndexFlatL2 (para precisión)**
    index_gt = faiss.IndexFlatL2(d)
    index_gt.add(embeddings)
    _, I_gt = index_gt.search(query_embeddings, k=10)  
    
    # **Medir tiempo de búsqueda y calcular precisión**
    start_time = time.time()
    _, I = index.search(query_embeddings, k=10)  
    search_time = time.time() - start_time

    # **Calcular Precisión**
    def precision_at_k(I_pred, I_gt, k=10):
        correct = 0
        for i in range(len(I_pred)):
            correct += len(set(I_pred[i]) & set(I_gt[i])) / k
        return correct / len(I_pred)

    precision = precision_at_k(I, I_gt, k=10)
    
    # Mostrar resultados
    print(f"Índice: {index_type}")
    print(f"Precisión: {precision:.4f}")
    print(f"Tiempo de búsqueda: {search_time:.4f} segundos")
    
    return precision  # Queremos maximizar la precisión

# 🔹 **Crear y optimizar el estudio con Optuna:**
study = optuna.create_study(direction="maximize")  # Maximizar precisión
study.optimize(objective, n_trials=50)  # Número de intentos

# Imprimir los mejores hiperparámetros y resultados
print("Mejores hiperparámetros: ", study.best_params)
print("Mejor precisión: ", study.best_value)


[I 2025-02-12 17:09:44,501] A new study created in memory with name: no-name-3ee1ee82-418c-4ba2-b9fc-528149e4ddfa
[I 2025-02-12 17:09:54,056] Trial 0 finished with value: 0.0034000000000000015 and parameters: {'index_type': 'IVFPQ', 'nlist': 812, 'm': 6, 'n_bits': 9}. Best is trial 0 with value: 0.0034000000000000015.


Índice: IVFPQ
Precisión: 0.0034
Tiempo de búsqueda: 0.0080 segundos


[I 2025-02-12 17:22:19,897] Trial 1 finished with value: 0.019900000000000008 and parameters: {'index_type': 'IVFPQ', 'nlist': 262, 'm': 128, 'n_bits': 16}. Best is trial 1 with value: 0.019900000000000008.


Índice: IVFPQ
Precisión: 0.0199
Tiempo de búsqueda: 3.2568 segundos


[I 2025-02-12 17:22:52,791] Trial 2 finished with value: 0.005999999999999995 and parameters: {'index_type': 'IVFPQ', 'nlist': 263, 'm': 6, 'n_bits': 12}. Best is trial 1 with value: 0.019900000000000008.


...

Índice: HNSW
Precisión: 0.8840
Tiempo de búsqueda: 0.9228 segundos


[I 2025-02-12 19:00:57,924] Trial 48 finished with value: 0.7087000000000013 and parameters: {'index_type': 'HNSW', 'M': 42, 'efConstruction': 385, 'efSearch': 179}. Best is trial 31 with value: 0.9071999999999925.


Índice: HNSW
Precisión: 0.7087
Tiempo de búsqueda: 0.4249 segundos


[I 2025-02-12 19:03:02,186] Trial 49 finished with value: 0.8620999999999933 and parameters: {'index_type': 'HNSW', 'M': 118, 'efConstruction': 363, 'efSearch': 163}. Best is trial 31 with value: 0.9071999999999925.


Índice: HNSW
Precisión: 0.8621
Tiempo de búsqueda: 0.8990 segundos
Mejores hiperparámetros:  {'index_type': 'HNSW', 'M': 128, 'efConstruction': 397, 'efSearch': 200}
Mejor precisión:  0.9071999999999925


In [16]:
def objective(trial):
    nlist = trial.suggest_int('nlist', 50, 500)
    m_choices = [8, 12, 16, 24, 32]
    m = trial.suggest_categorical('m', m_choices)

    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
    index.train(embeddings)
    index.add(embeddings)

    start_time = time.time()
    distances, indices = index.search(query_embeddings, k=10)
    search_time = time.time() - start_time

    # Convertimos distancias en similaridades
    similarity_scores = -distances.mean()  # Faiss devuelve distancias L2, así que las invertimos

    return similarity_scores  # Queremos MAXIMIZAR la similitud

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f"Mejores hiperparámetros: {study.best_params}")


[I 2025-02-12 15:07:22,408] A new study created in memory with name: no-name-9010cd00-5584-42ad-843f-2b830bb0d6e6
c:\Users\Carlo\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_core\_methods.py:135: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
[I 2025-02-12 15:07:27,781] Trial 0 finished with value: -inf and parameters: {'nlist': 468, 'm': 16}. Best is trial 0 with value: -inf.
[I 2025-02-12 15:07:31,038] Trial 1 finished with value: -inf and parameters: {'nlist': 428, 'm': 32}. Best is trial 0 with value: -inf.
[I 2025-02-12 15:07:34,297] Trial 2 finished with value: -127.42923736572266 and parameters: {'nlist': 367, 'm': 8}. Best is trial 2 with value: -127.42923736572266.
[I 2025-02-12 15:07:36,841] Trial 3 finished with value: -127.21797180175781 and parameters: {'nlist': 226, 'm': 32}. Best is trial 3 with value: -127.21797180175781.
[I 2025-02-12 15:07:40,306] Trial 4 finished with value: -inf and pa

Mejores hiperparámetros: {'nlist': 50, 'm': 24}


In [ ]:
import numpy as np
import faiss
import time

# Dimensiones de los embeddings
d = 384  

# Número de embeddings (películas)
n = 141057

embeddings

# 🔹 Seleccionar 30 películas al azar para recomendar similares
nq = 30  # Número de consultas
query_indices = np.random.choice(n, nq, replace=False)  # Índices aleatorios de películas
query_embeddings = embeddings[query_indices]  # Obtener los embeddings de esas películas

# 🔹 **Índice FlatL2 (Ground Truth)**
index_l2 = faiss.IndexFlatL2(d)
index_l2.add(embeddings)

# Buscar los 10 vecinos más cercanos con el índice exacto
_, I_gt = index_l2.search(query_embeddings, k=10)

# 🔹 **Índice 1: IVFPQ**
nlist = 100  # Número de listas
m = 16       # Número de divisiones
n_bits = 8   # Bits por subvector

quantizer = faiss.IndexFlatL2(d)
index_ivfpq = faiss.IndexIVFPQ(quantizer, d, nlist, m, n_bits)
index_ivfpq.train(embeddings)

# Añadir embeddings en lotes
batch_size = 10000
for i in range(0, n, batch_size):
    index_ivfpq.add(embeddings[i:i + batch_size])

# 🔹 **Índice 2: HNSW**
M = 128
efConstruction = 397
efSearch = 64

index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.hnsw.efConstruction = efConstruction
index_hnsw.hnsw.efSearch = efSearch

# Añadir embeddings en lotes
for i in range(0, n, batch_size):
    index_hnsw.add(embeddings[i:i + batch_size])

# 🔹 **Función para medir el tiempo de búsqueda**
def search_and_measure(index, queries, k=10):
    start_time = time.time()
    _, I = index.search(queries, k)  # Buscar los K más similares
    total_time = time.time() - start_time
    avg_time_per_query = total_time / len(queries)
    return I, total_time, avg_time_per_query

# 🔹 **Buscar películas similares con IVFPQ**
I_ivfpq, time_ivfpq, avg_time_ivfpq = search_and_measure(index_ivfpq, query_embeddings, k=10)

# 🔹 **Buscar películas similares con HNSW**
I_hnsw, time_hnsw, avg_time_hnsw = search_and_measure(index_hnsw, query_embeddings, k=10)

# 🔹 **Función para calcular precisión**
def precision_at_k(I_pred, I_gt, k=10):
    correct = 0
    for i in range(len(I_pred)):
        correct += len(set(I_pred[i]) & set(I_gt[i])) / k
    return correct / len(I_pred)

# 🔹 **Calcular precisión**
precision_ivfpq = precision_at_k(I_ivfpq, I_gt, k=10)
precision_hnsw = precision_at_k(I_hnsw, I_gt, k=10)

# 🔹 **Buscar películas similares con FlatL2**
I_l2, time_l2, avg_time_l2 = search_and_measure(index_l2, query_embeddings, k=10)

# 🔹 **Resultados**
print("\n🔹 Resultados de las 30 consultas:")
print(f"L2    - Precisión: 1.0000 | Tiempo total: {time_l2:.4f} s | Tiempo por recomendación: {avg_time_l2:.4f} s")
print(f"IVFPQ - Precisión: {precision_ivfpq:.4f} | Tiempo total: {time_ivfpq:.4f} s | Tiempo por recomendación: {avg_time_ivfpq:.4f} s")
print(f"HNSW  - Precisión: {precision_hnsw:.4f} | Tiempo total: {time_hnsw:.4f} s | Tiempo por recomendación: {avg_time_hnsw:.4f} s")

# 🔹 **Mostrar algunas recomendaciones**
print("\nEjemplo de recomendaciones para la primera consulta:")
print("L2   :", I_gt[0])
print("IVFPQ:", I_ivfpq[0])
print("HNSW :", I_hnsw[0])



🔹 Resultados de las 30 consultas:
L2    - Precisión: 1.0000 | Tiempo total: 0.0441 s | Tiempo por recomendación: 0.0015 s
IVFPQ - Precisión: 0.3033 | Tiempo total: 0.0006 s | Tiempo por recomendación: 0.0000 s
HNSW  - Precisión: 0.9967 | Tiempo total: 0.0048 s | Tiempo por recomendación: 0.0002 s

Ejemplo de recomendaciones para la primera consulta:
L2   : [  6720  66615  69161  71080 137434  42290  17425  25521  82429 111461]
IVFPQ: [  6720  66615  71080  69161  58065  74932  82429 111461 137434  39320]
HNSW : [  6720  66615  69161  71080 137434  42290  17425  25521  82429 111461]


In [33]:
import numpy as np
import faiss
import time

# Dimensiones de los embeddings
d = 384  

# Número de embeddings (películas)
n = 141057

# Supongamos que ya tienes tus embeddings cargados
# embeddings

# 🔹 Seleccionar 30 películas al azar para recomendar similares
nq = 30  # Número de consultas
query_indices = np.random.choice(n, nq, replace=False)  # Índices aleatorios de películas
query_embeddings = embeddings[query_indices]  # Obtener los embeddings de esas películas

# 🔹 **Índice FlatL2 (Ground Truth)**
index_l2 = faiss.IndexFlatL2(d)
index_l2.add(embeddings)

# Buscar los 10 vecinos más cercanos con el índice exacto
_, I_gt = index_l2.search(query_embeddings, k=10)

# 🔹 **Índice 1: IVFPQ**
nlist = 100  # Número de listas
m = 16       # Número de divisiones
n_bits = 8   # Bits por subvector

quantizer = faiss.IndexFlatL2(d)
index_ivfpq = faiss.IndexIVFPQ(quantizer, d, nlist, m, n_bits)
index_ivfpq.train(embeddings)

# Añadir embeddings en lotes
batch_size = 10000
for i in range(0, n, batch_size):
    index_ivfpq.add(embeddings[i:i + batch_size])

# 🔹 **Índice 2: HNSW**
M = 128
efConstruction = 397
efSearch = 64

index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.hnsw.efConstruction = efConstruction
index_hnsw.hnsw.efSearch = efSearch

# Añadir embeddings en lotes
for i in range(0, n, batch_size):
    index_hnsw.add(embeddings[i:i + batch_size])

# 🔹 **Función para medir el tiempo de búsqueda**
def search_and_measure(index, queries, k=10):
    start_time = time.time()
    _, I = index.search(queries, k)  # Buscar los K más similares
    total_time = time.time() - start_time
    avg_time_per_query = total_time / len(queries)
    return I, total_time, avg_time_per_query

# 🔹 **Buscar películas similares con IVFPQ**
I_ivfpq, time_ivfpq, avg_time_ivfpq = search_and_measure(index_ivfpq, query_embeddings, k=10)

# 🔹 **Buscar películas similares con HNSW**
I_hnsw, time_hnsw, avg_time_hnsw = search_and_measure(index_hnsw, query_embeddings, k=10)

# 🔹 **Función para filtrar las recomendaciones y evitar las películas consultadas**
def filter_recommendations(I, query_indices, k=10):
    for i in range(len(I)):
        # Filtrar las películas consultadas
        filtered = [x for x in I[i] if x not in query_indices]
        
        # Si hay menos de k resultados después de filtrar, agregar más recomendaciones
        remaining = k - len(filtered)
        if remaining > 0:
            additional_recommendations = np.random.choice(n, remaining, replace=False).tolist()  # Genera recomendaciones aleatorias
            filtered += additional_recommendations  # Añadir las recomendaciones faltantes
        
        # Si después de todo esto aún no hay 10 recomendaciones, ajustamos el tamaño
        I[i] = filtered[:k]  # Limitar a 10 recomendaciones

    return I

# 🔹 **Filtrar las recomendaciones**
I_ivfpq = filter_recommendations(I_ivfpq, query_indices)
I_hnsw = filter_recommendations(I_hnsw, query_indices)

# 🔹 **Función para calcular precisión**
def precision_at_k(I_pred, I_gt, k=10):
    correct = 0
    for i in range(len(I_pred)):
        correct += len(set(I_pred[i]) & set(I_gt[i])) / k
    return correct / len(I_pred)

# 🔹 **Calcular precisión**
precision_ivfpq = precision_at_k(I_ivfpq, I_gt, k=10)
precision_hnsw = precision_at_k(I_hnsw, I_gt, k=10)

# 🔹 **Buscar películas similares con FlatL2**
I_l2, time_l2, avg_time_l2 = search_and_measure(index_l2, query_embeddings, k=10)

# 🔹 **Resultados**
print("\n🔹 Resultados de las 30 consultas:")
print(f"L2    - Precisión: 1.0000 | Tiempo total: {time_l2:.4f} s | Tiempo por recomendación: {avg_time_l2:.4f} s")
print(f"IVFPQ - Precisión: {precision_ivfpq:.4f} | Tiempo total: {time_ivfpq:.4f} s | Tiempo por recomendación: {avg_time_ivfpq:.4f} s")
print(f"HNSW  - Precisión: {precision_hnsw:.4f} | Tiempo total: {time_hnsw:.4f} s | Tiempo por recomendación: {avg_time_hnsw:.4f} s")

# 🔹 **Mostrar algunas recomendaciones**
print("\nEjemplo de recomendaciones para la primera consulta:")
print("L2   :", I_gt[0])
print("IVFPQ:", I_ivfpq[0])
print("HNSW :", I_hnsw[0])



🔹 Resultados de las 30 consultas:
L2    - Precisión: 1.0000 | Tiempo total: 0.0528 s | Tiempo por recomendación: 0.0018 s
IVFPQ - Precisión: 0.1500 | Tiempo total: 0.0000 s | Tiempo por recomendación: 0.0000 s
HNSW  - Precisión: 0.9000 | Tiempo total: 0.0050 s | Tiempo por recomendación: 0.0002 s

Ejemplo de recomendaciones para la primera consulta:
L2   : [ 31391  79172 108204  22371  88525 118232  78155 107187  75444   7005]
IVFPQ: [133667 101423 131055 137393  45367  19864  25717  14471  80115 124809]
HNSW : [ 79172 108204  22371  88525 118232  78155 107187  75444   7005  42905]


In [9]:
# Dimensiones de los embeddings (d)
d = 384

# Número de embeddings (n)
n = 141057

# Asume que 'embeddings' es tu matriz de vectores (películas)
# embeddings = ...

# Hiperparámetros de HNSW
M = 128           # Número de conexiones por capa
efConstruction = 397  # Parámetro para la construcción del índice (cuanto mayor, mejor precisión pero más lento)
efSearch = 64      # Parámetro para la búsqueda (cuanto mayor, más preciso)

# Crear el índice HNSW
index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.hnsw.efConstruction = efConstruction
index_hnsw.hnsw.efSearch = efSearch

# Añadir los embeddings al índice
index_hnsw.add(embeddings)

# Guardar el índice en un archivo
faiss.write_index(index_hnsw, 'index_hnsw.idx')

print("Índice HNSW guardado como 'index_hnsw.idx'")


Índice HNSW guardado como 'index_hnsw.idx'
